In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from typing import Dict, Text
from tensorflow import keras
from tqdm.notebook import tqdm
tqdm.pandas()
import tensorflow_recommenders as tfrs
try:
    if not tf.config.list_physical_devices('GPU'):
        assert tf.__version__ >= "2.0"
        print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
        if IS_COLAB:
            print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
        if IS_KAGGLE:
            print("Go to Settings > Accelerator and select GPU.")
except:
    if not tf.test.is_gpu_available():
        assert tf.__version__ >= "2.0"
        print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
        if IS_COLAB:
            print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
        if IS_KAGGLE:
            print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import pandas as pd
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import datetime as dt
from pathlib import Path

# to make this notebook's output stable across runs
np.random.seed(42)
try:
    if not tf.config.list_physical_devices('GPU'):
        tf.random.set_seed(42)
    else:
        tf.random.set_random_seed(42)
except:
    if not tf.test.is_gpu_available():
        tf.random.set_seed(42)
    else:
        tf.random.set_random_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


In [2]:
def zero_f(item):
    item=str(item)
    tem=len(item)
    if(len(item)<10):
        item=item.zfill(10)
    return item

In [3]:
trans_train = pd.read_csv('h-and-m-personalized-fashion-recommendations/transactions_train.csv',dtype={'customer_id': str,'article_id':str})
trans_train['quantity']=1
master_df = trans_train[['customer_id','article_id','quantity']].astype(str)
master_df['article_id']=master_df['article_id'].apply(zero_f)
master_df['quantity'] = master_df['quantity'].astype(float)
masterdf = master_df
master_df.head()

,customer_id,article_id,quantity
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,1.0
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,1.0
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,1.0
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,1.0
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,1.0


In [4]:
### define interactions data and user data

### interactions 
### here we create a reference table of the user , item, and quantity purchased
interactions_dict = masterdf.groupby(['customer_id', 'article_id'])[ 'quantity'].sum().reset_index()

## we tansform the table inta a dictionary , which then we feed into tensor slices
# this step is crucial as this will be the type of data fed into the embedding layers
interactions_dict = {name: np.array(value) for name, value in interactions_dict.items()}
interactions = tf.data.Dataset.from_tensor_slices(interactions_dict)

## we do similar step for item, where this is the reference table for items to be recommended
items_dict = masterdf[['article_id']].drop_duplicates()
items_dict = {name: np.array(value) for name, value in items_dict.items()}
items = tf.data.Dataset.from_tensor_slices(items_dict)

## map the features in interactions and items to an identifier that we will use throught the embedding layers
## do it for all the items in interaction and item table
## you may often get itemtype error, so that is why here i am casting the quantity type as float to ensure consistency
interactions = interactions.map(lambda x: {
    'customer_id' : x['customer_id'], 
    'article_id' : x['article_id'], 
    'quantity' : float(x['quantity']),

})

items = items.map(lambda x: x['article_id'])

In [5]:
unique_item_titles = np.unique(np.concatenate(list(items.batch(1_000))))
unique_user_ids = np.unique(np.concatenate(list(interactions.batch(1_000_000).map(lambda x: x["customer_id"]))))

In [6]:
### get unique item and user id's as a lookup table
unique_item_titles = np.unique(np.concatenate(list(items.batch(1_000))))
unique_user_ids = np.unique(np.concatenate(list(interactions.batch(1_000_000).map(lambda x: x["customer_id"]))))

# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = interactions.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [7]:
class RetailModel(tfrs.Model):

    def __init__(self, user_model, item_model):
        super().__init__()
        
        self.item_model: tf.keras.Model = item_model
        self.user_model: tf.keras.Model = user_model
        
        ### for retrieval model. we take top-k accuracy as metrics
        metrics = tfrs.metrics.FactorizedTopK(candidates=items.batch(256).map(item_model))
        
        # define the task, which is retrieval                                    )    
        task = tfrs.tasks.Retrieval(metrics=metrics)
       
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["customer_id"])
        # And pick out the movie features and pass them into the movie model,
        # getting embeddings back.
        positive_movie_embeddings = self.item_model(features["article_id"])

        # The task computes the loss and the metrics.
        return self.task(user_embeddings, positive_movie_embeddings)

In [8]:
### Fitting and evaluating

### we choose the dimensionality of the query and candicate representation.
embedding_dimension = 256

## we pass the model, which is the same model we created in the query and candidate tower, into the model
item_model = tf.keras.Sequential([
                                tf.keras.layers.experimental.preprocessing.StringLookup(
                                vocabulary=unique_item_titles, mask_token=None),
                                tf.keras.layers.Embedding(len(unique_item_titles) + 1, embedding_dimension)
                                ])

user_model = tf.keras.Sequential([
                                tf.keras.layers.experimental.preprocessing.StringLookup(
                                vocabulary=unique_user_ids, mask_token=None),
                                # We add an additional embedding to account for unknown tokens.
                                tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
                                ])

model = RetailModel(user_model, item_model)

# a smaller learning rate may make the model move slower and prone to overfitting, so we stick to 0.1
# other optimizers, such as SGD and Adam, are listed here https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
model.compile(optimizer=tf.keras.optimizers.SGD())

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

## fit the model with ten epochs
model_hist = model.fit(cached_train, epochs=20)

#evaluate the model
model.evaluate(cached_test, return_dict=True)

Epoch 1/20
10/10 [==============================] - 121s 11s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.2500e-05 - factorized_top_k/top_10_categorical_accuracy: 5.0000e-05 - factorized_top_k/top_50_categorical_accuracy: 5.1250e-04 - factorized_top_k/top_100_categorical_accuracy: 9.7500e-04 - loss: 70368.6172 - regularization_loss: 0.0000e+00 - total_loss: 70368.6172
Epoch 2/20
10/10 [==============================] - 115s 11s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 8.7500e-05 - factorized_top_k/top_10_categorical_accuracy: 2.2500e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0014 - factorized_top_k/top_100_categorical_accuracy: 0.0026 - loss: 70335.1420 - regularization_loss: 0.0000e+00 - total_loss: 70335.1420
Epoch 3/20
10/10 [==============================] - 115s 11s/step - factorized_top_k/top_1_categorical_accuracy: 1.2500e-05 - factor

5/5 [==============================] - 30s 5s/step - factorized_top_k/top_1_categorical_accuracy: 1.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0011 - factorized_top_k/top_10_categorical_accuracy: 0.0019 - factorized_top_k/top_50_categorical_accuracy: 0.0052 - factorized_top_k/top_100_categorical_accuracy: 0.0078 - loss: 32596.1426 - regularization_loss: 0.0000e+00 - total_loss: 32596.1426


{'factorized_top_k/top_1_categorical_accuracy': 9.999999747378752e-05,
 'factorized_top_k/top_5_categorical_accuracy': 0.0010999999940395355,
 'factorized_top_k/top_10_categorical_accuracy': 0.0018500000005587935,
 'factorized_top_k/top_50_categorical_accuracy': 0.005200000014156103,
 'factorized_top_k/top_100_categorical_accuracy': 0.007849999703466892,
 'loss': 29636.07421875,
 'regularization_loss': 0,
 'total_loss': 29636.07421875}

In [9]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 28s 5s/step - factorized_top_k/top_1_categorical_accuracy: 1.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0011 - factorized_top_k/top_10_categorical_accuracy: 0.0019 - factorized_top_k/top_50_categorical_accuracy: 0.0052 - factorized_top_k/top_100_categorical_accuracy: 0.0078 - loss: 32596.1426 - regularization_loss: 0.0000e+00 - total_loss: 32596.1426


{'factorized_top_k/top_1_categorical_accuracy': 9.999999747378752e-05,
 'factorized_top_k/top_5_categorical_accuracy': 0.0010999999940395355,
 'factorized_top_k/top_10_categorical_accuracy': 0.0018500000005587935,
 'factorized_top_k/top_50_categorical_accuracy': 0.005200000014156103,
 'factorized_top_k/top_100_categorical_accuracy': 0.007849999703466892,
 'loss': 29636.07421875,
 'regularization_loss': 0,
 'total_loss': 29636.07421875}

In [10]:
unique_item_titles

array([b'0108775015', b'0108775044', b'0108775051', ..., b'0953450001',
       b'0953763001', b'0956217002'], dtype=object)

In [11]:
unique_user_ids 

array([b'00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657',
       b'0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa',
       b'000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318',
       ...,
       b'ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1778d0116cffd259264',
       b'ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38b2236865d949d4df6a',
       b'ffffd9ac14e89946416d80e791d064701994755c3ab686a1eaf3458c36f52241'],
      dtype=object)

In [12]:
train

<TakeDataset shapes: {customer_id: (), article_id: (), quantity: ()}, types: {customer_id: tf.string, article_id: tf.string, quantity: tf.float32}>

In [13]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(items.batch(100).map(lambda items: (items,model.item_model(items))))

In [14]:
j = '000231cc9af9e58ab4edc66fbd61da921b144ba85bc1c00d0ae2309531e4c210'
_, titles = index(tf.constant([j]),k=12)
print(f"Recommendations for user %s: {titles[0]}" %(j))

Recommendations for user 000231cc9af9e58ab4edc66fbd61da921b144ba85bc1c00d0ae2309531e4c210: [b'0572797002' b'0686564004' b'0670698003' b'0682670002' b'0686331005'
 b'0634013022' b'0841383003' b'0695632001' b'0798577002' b'0733267001'
 b'0772281001' b'0778064003']


In [15]:
def run_f(item):
    _, titles = index(tf.constant([item]),k=12)
    t = np.array(titles[0])
    l = [el.decode('UTF-8') for el in t]
    l = " ".join(l)
    return l

In [ ]:
submission_file = pd.read_csv('h-and-m-personalized-fashion-recommendations/sample_submission.csv')
sub_cust = submission_file["customer_id"]
sub_df = pd.DataFrame(columns=['Customer_Id', 'Article_Id'])
submission_file["prediction"] = submission_file['customer_id'].progress_apply(run_f)
submission_file

  0%|          | 0/1371980 [00:00<?, ?it/s]

In [ ]:
submission_file.to_csv('submission.csv',index=False)